# 重新定义大模型和智能体

## IO：基于标准输入输出的流式调用

In [1]:
from textlong.io import TextBlock, stream_log

In [2]:
def hello():
    yield TextBlock("info", "hello textlong")
    yield TextBlock("chunk", "Textlong")
    yield TextBlock("chunk", "很")
    yield TextBlock("chunk", "棒！")

stream_log(hello)

>-[INFO]>> hello textlong
Textlong很棒！



{'output': 'Textlong很棒！', 'tools_call': {}}

## 调用大模型

### 大模型能力：通义千问

In [3]:
from textlong.llm.dashscope import qwen
from textlong.io import stream_log

stream_log(
    qwen,
    [{"role": "user", "content": "你能帮我写一首关于兔子做梦的四句儿歌?"}],
    model="qwen2-1.5b-instruct"
)

小兔子，睡梦乡，  
梦见自己穿花裳。  
耳朵长，蹦蹦跳，  
月光下，梦儿飘。



{'output': '小兔子，睡梦乡，  \n梦见自己穿花裳。  \n耳朵长，蹦蹦跳，  \n月光下，梦儿飘。', 'tools_call': {}}

### fake_llm: 模拟大模型调用过程

这可以观察提示语等内部结构。

In [4]:
from textlong.llm import fake_llm
from textlong.io import stream_log

stream_log(
    fake_llm,
    [
        {"role": "system", "content": "你是一个写作能手。"},
        {"role": "user", "content": "你能帮我写一首关于兔子做梦的四句儿歌?"}
    ],
    model="qwen2-1.5b-instruct"
)

>-[INFO]>> system: 你是一个写作能手。
>-[INFO]>> user: 你能帮我写一首关于兔子做梦的四句儿歌?
这是一个模拟调用!



{'output': '这是一个模拟调用!', 'tools_call': {}}

### 多轮对话

使用`messages`管理多轮对话的记忆。<br>
你也可以在`chat`函数中通过参数`k`限制希望保留的对话轮次，但这不影响`messages`记录完整的对话过程。

In [1]:
from textlong.llm import chat, fake_llm
from textlong.llm.dashscope import qwen
messages = []

In [2]:
chat(qwen, "你能帮我写一首关于兔子做梦的四句儿歌?", messages)

小兔子，睡梦乡，  
梦见自己穿花裳。  
耳朵长，尾巴摇，  
月亮船上蹦蹦跳。



'小兔子，睡梦乡，  \n梦见自己穿花裳。  \n耳朵长，尾巴摇，  \n月亮船上蹦蹦跳。'

In [3]:
chat(qwen, "换成小羊吧", messages)

小羊羔，梦中游，  
云朵床，轻轻悠。  
羊毛软，咩咩笑，  
星空下，舞步巧。



'小羊羔，梦中游，  \n云朵床，轻轻悠。  \n羊毛软，咩咩笑，  \n星空下，舞步巧。'

In [4]:
messages

[{'role': 'user', 'content': '你能帮我写一首关于兔子做梦的四句儿歌?'},
 {'role': 'assistant',
  'content': '小兔子，睡梦乡，  \n梦见自己穿花裳。  \n耳朵长，尾巴摇，  \n月亮船上蹦蹦跳。'},
 {'role': 'user', 'content': '换成小羊吧'},
 {'role': 'assistant',
  'content': '小羊羔，梦中游，  \n云朵床，轻轻悠。  \n羊毛软，咩咩笑，  \n星空下，舞步巧。'}]

In [5]:
chat(qwen, "改为在蓝天上", messages, k=1)

翱翔在蓝天上，白云悠悠，阳光明媚，仿佛整个世界都变得无限宽广。鸟儿们自由地穿梭在碧空之中，留下一串串悠扬的歌声，它们的影子在广袤的大地上轻轻掠过，绘出一幅幅生动的画卷。飞机划过天际，留下一条长长的尾云，像是天空的轨迹，记录着人类探索的渴望与梦想。在这样的蓝天下，人们仰望，不仅能看到自然的美好，更能感受到心灵的宁静与自由，仿佛所有的烦恼都随风而去，只剩下纯粹与宁静伴随着每一次深呼吸。



'翱翔在蓝天上，白云悠悠，阳光明媚，仿佛整个世界都变得无限宽广。鸟儿们自由地穿梭在碧空之中，留下一串串悠扬的歌声，它们的影子在广袤的大地上轻轻掠过，绘出一幅幅生动的画卷。飞机划过天际，留下一条长长的尾云，像是天空的轨迹，记录着人类探索的渴望与梦想。在这样的蓝天下，人们仰望，不仅能看到自然的美好，更能感受到心灵的宁静与自由，仿佛所有的烦恼都随风而去，只剩下纯粹与宁静伴随着每一次深呼吸。'

In [6]:
chat(fake_llm, "改为在草地上", messages, k=10)

>-[INFO]>> user: 你能帮我写一首关于兔子做梦的四句儿歌?
>-[INFO]>> assistant: 小兔子，睡梦乡，  
梦见自己穿花裳。  
耳朵长，尾巴摇，  
月亮船上蹦蹦跳。
>-[INFO]>> user: 换成小羊吧
>-[INFO]>> assistant: 小羊羔，梦中游，  
云朵床，轻轻悠。  
羊毛软，咩咩笑，  
星空下，舞步巧。
>-[INFO]>> user: 改为在蓝天上
>-[INFO]>> assistant: 翱翔在蓝天上，白云悠悠，阳光明媚，仿佛整个世界都变得无限宽广。鸟儿们自由地穿梭在碧空之中，留下一串串悠扬的歌声，它们的影子在广袤的大地上轻轻掠过，绘出一幅幅生动的画卷。飞机划过天际，留下一条长长的尾云，像是天空的轨迹，记录着人类探索的渴望与梦想。在这样的蓝天下，人们仰望，不仅能看到自然的美好，更能感受到心灵的宁静与自由，仿佛所有的烦恼都随风而去，只剩下纯粹与宁静伴随着每一次深呼吸。
>-[INFO]>> user: 改为在草地上
这是一个模拟调用!



'这是一个模拟调用!'

In [7]:
messages

[{'role': 'user', 'content': '你能帮我写一首关于兔子做梦的四句儿歌?'},
 {'role': 'assistant',
  'content': '小兔子，睡梦乡，  \n梦见自己穿花裳。  \n耳朵长，尾巴摇，  \n月亮船上蹦蹦跳。'},
 {'role': 'user', 'content': '换成小羊吧'},
 {'role': 'assistant',
  'content': '小羊羔，梦中游，  \n云朵床，轻轻悠。  \n羊毛软，咩咩笑，  \n星空下，舞步巧。'},
 {'role': 'user', 'content': '改为在蓝天上'},
 {'role': 'assistant',
  'content': '翱翔在蓝天上，白云悠悠，阳光明媚，仿佛整个世界都变得无限宽广。鸟儿们自由地穿梭在碧空之中，留下一串串悠扬的歌声，它们的影子在广袤的大地上轻轻掠过，绘出一幅幅生动的画卷。飞机划过天际，留下一条长长的尾云，像是天空的轨迹，记录着人类探索的渴望与梦想。在这样的蓝天下，人们仰望，不仅能看到自然的美好，更能感受到心灵的宁静与自由，仿佛所有的烦恼都随风而去，只剩下纯粹与宁静伴随着每一次深呼吸。'},
 {'role': 'user', 'content': '改为在草地上'},
 {'role': 'assistant', 'content': '这是一个模拟调用!'}]

### 在大模型中支持回调工具

仍然采纳`langchain`来定义工具，这可以大大简化回调工具的管理。

涉及定义工具的技巧主要包括：
- 使用 tool 装饰函数将新函数定义为工具类型
- 使用 StructTool 转换已有的函数为工具类型
- 使用 BaseModel 补充工具的参数定义

采用以上方法定义好工具，就可以使用 convert_to_openai_tool 将其转换为官方要求的 JSON 结构（确实会比手写节省很多精力）。

In [8]:
from textlong.llm.tools import tool, convert_to_openai_tool
from textlong.llm.dashscope import qwen
from textlong.io import stream_log
import json

@tool
def get_current_weather(location: str):
    """获取城市的天气情况"""
    return f"{location}今天是晴天。 "

stream_log(qwen, "今天广州天气如何啊", tools=[convert_to_openai_tool(get_current_weather)])

>-[TOOLS_CALL]>> {"0": {"index": 0, "id": "", "type": "function", "function": {"name": "get_current_weather", "arguments": "{\"location\": \"\u5e7f\u5dde\"}"}}}


{'output': '',
 'tools_call': {0: {'index': 0,
   'id': '',
   'type': 'function',
   'function': {'name': 'get_current_weather',
    'arguments': '{"location": "广州"}'}}}}

<div class="alert alert-info">
⚠️ 上面的调用仅仅是返回了工具描述，尚未真正执行工具代码。
</div>

### 在多轮对话中支持回调工具

In [9]:
from textlong.llm.tools import tool, convert_to_openai_tool
from textlong.llm.dashscope import qwen
from textlong.llm import chat
import json

@tool
def get_current_weather(location: str):
    """获取城市的天气情况"""
    return f"{location}今天是晴天。 "

messages = []
chat(qwen, "今天广州天气如何啊", messages, tools=[convert_to_openai_tool(get_current_weather)], toolkits=[get_current_weather])

>-[TOOLS_CALL]>> {"0": {"index": 0, "id": "", "type": "function", "function": {"name": "get_current_weather", "arguments": "{\"location\": \"\u5e7f\u5dde\"}"}}}
广州今天是晴天。不过，请注意实时天气可能会有所变化，出行记得查看最新的天气预报。



'广州今天是晴天。不过，请注意实时天气可能会有所变化，出行记得查看最新的天气预报。'

In [10]:
chat(qwen, "这是怎么得出来的？", messages, tools=[convert_to_openai_tool(get_current_weather)], toolkits=[get_current_weather])

我使用了天气查询工具来获取信息。工具接收到请求后，会从它的数据源中查找广州当前的天气状况，并返回结果给我。根据查询结果，我得知广州今天是晴天。这种方式可以提供实时或接近实时的天气信息。



'我使用了天气查询工具来获取信息。工具接收到请求后，会从它的数据源中查找广州当前的天气状况，并返回结果给我。根据查询结果，我得知广州今天是晴天。这种方式可以提供实时或接近实时的天气信息。'

In [11]:
chat(qwen, "再看看上海", messages, tools=[convert_to_openai_tool(get_current_weather)], toolkits=[get_current_weather])

>-[TOOLS_CALL]>> {"0": {"index": 0, "id": "", "type": "function", "function": {"name": "get_current_weather", "arguments": "{\"location\": \"\u4e0a\u6d77\"}"}}}
上海今天的天气是晴朗的。



'上海今天的天气是晴朗的。'

In [12]:
messages

[{'role': 'user', 'content': '今天广州天气如何啊'},
 {'role': 'assistant',
  'content': '',
  'tool_calls': [{'index': 0,
    'id': '',
    'type': 'function',
    'function': {'name': 'get_current_weather',
     'arguments': '{"location": "广州"}'}}]},
 {'role': 'tool', 'name': 'get_current_weather', 'content': '广州今天是晴天。 '},
 {'role': 'assistant', 'content': '广州今天是晴天。不过，请注意实时天气可能会有所变化，出行记得查看最新的天气预报。'},
 {'role': 'user', 'content': '这是怎么得出来的？'},
 {'role': 'assistant',
  'content': '我使用了天气查询工具来获取信息。工具接收到请求后，会从它的数据源中查找广州当前的天气状况，并返回结果给我。根据查询结果，我得知广州今天是晴天。这种方式可以提供实时或接近实时的天气信息。'},
 {'role': 'user', 'content': '再看看上海'},
 {'role': 'assistant',
  'content': '',
  'tool_calls': [{'index': 0,
    'id': '',
    'type': 'function',
    'function': {'name': 'get_current_weather',
     'arguments': '{"location": "上海"}'}}]},
 {'role': 'tool', 'name': 'get_current_weather', 'content': '上海今天是晴天。 '},
 {'role': 'assistant', 'content': '上海今天的天气是晴朗的。'}]

## 写作提示语模板

In [13]:
from textlong.hub import load_chat_template, load_resource_chat_template

### 加载对话模板的资源

指定资源库中的模板ID即可加载模板。

In [14]:
load_chat_template("IDEA")

PromptTemplate(input_variables=['task'], template='你是强大的写作助手。\n\n你必须遵循以下约束来完成任务:\n1. 直接输出你的结果，不要评论，不要啰嗦\n2. 使用markdown格式输出\n\n**你的任务是:**\n{{task}}\n', template_format='mustache')

In [15]:
load_chat_template("OUTLINE")

PromptTemplate(input_variables=['task'], template='你是强大的写作助手。\n\n你必须遵循以下约束来完成任务:\n1. 直接输出你的结果，不要评论，不要啰嗦\n2. 使用markdown格式输出\n3. 请按照需要输出一份写作提纲\n4. 提纲的构成应当由多个阔系额要求构成\n5. 扩写要求必须包含清晰描述，如预估字数、创意要点、创作思路、创作中涉及到的实体名称等\n5. 每一个扩写要求由完整的 `<OUTLINE>`和`</OUTLINE>` 包裹\n6. 你只能输出提纲，不要输出具体的扩写内容\n\n\n**你的任务是:**\n{{task}}\n\n\n**输出例子1**\n```\n# 第一章 标题1\n## 一、XXXX\n<OUTLINE>\n扩写摘要:\n对扩写内容做摘要总结\n\n扩写要求：\n- xxx\n- xxx\n</OUTLINE>\n\n## 二、XXXX\n<OUTLINE>\n</OUTLINE>\n\n## （更多扩写提纲）\n```\n\n', template_format='mustache')

### 查看构造出的提示语模板

In [16]:
print(load_chat_template("OUTLINE").format(task="请帮我写一首歌"))

你是强大的写作助手。

你必须遵循以下约束来完成任务:
1. 直接输出你的结果，不要评论，不要啰嗦
2. 使用markdown格式输出
3. 请按照需要输出一份写作提纲
4. 提纲的构成应当由多个阔系额要求构成
5. 扩写要求必须包含清晰描述，如预估字数、创意要点、创作思路、创作中涉及到的实体名称等
5. 每一个扩写要求由完整的 `<OUTLINE>`和`</OUTLINE>` 包裹
6. 你只能输出提纲，不要输出具体的扩写内容


**你的任务是:**
请帮我写一首歌


**输出例子1**
```
# 第一章 标题1
## 一、XXXX
<OUTLINE>
扩写摘要:
对扩写内容做摘要总结

扩写要求：
- xxx
- xxx
</OUTLINE>

## 二、XXXX
<OUTLINE>
</OUTLINE>

## （更多扩写提纲）
```




### 单轮写作

In [17]:
from textlong.llm import chat, write, fake_llm
from textlong.llm.dashscope import qwen
messages = []

In [18]:
write(qwen, "IDEA", {"task": "帮我写一首关于兔子的四句儿歌"}, messages)

小白兔，蹦蹦跳，  
耳朵长，尾巴短摇摇。  
爱吃萝卜和青菜，  
快快乐乐满地跑。



'小白兔，蹦蹦跳，  \n耳朵长，尾巴短摇摇。  \n爱吃萝卜和青菜，  \n快快乐乐满地跑。'

In [19]:
write(qwen, "IDEA", {"task": "帮我写一首关于小马的四句儿歌"}, messages)

小马儿小马儿，蹄儿踏踏响，
草原上奔跑，快乐无边疆。
尾巴摇摇，鬃毛闪闪亮，
朋友遍天下，笑声多欢畅。



'小马儿小马儿，蹄儿踏踏响，\n草原上奔跑，快乐无边疆。\n尾巴摇摇，鬃毛闪闪亮，\n朋友遍天下，笑声多欢畅。'

In [20]:
messages

[{'role': 'system',
  'content': '你是强大的写作助手。\n\n你必须遵循以下约束来完成任务:\n1. 直接输出你的结果，不要评论，不要啰嗦\n2. 使用markdown格式输出\n\n**你的任务是:**\n帮我写一首关于小马的四句儿歌\n'},
 {'role': 'user', 'content': '请你开始'},
 {'role': 'assistant',
  'content': '小马儿小马儿，蹄儿踏踏响，\n草原上奔跑，快乐无边疆。\n尾巴摇摇，鬃毛闪闪亮，\n朋友遍天下，笑声多欢畅。'}]

### 单轮写作 + 多轮对话

In [21]:
chat(qwen, "改为8句吧", messages)

小马儿小马儿，清晨蹄声忙，
踏上绿茵茵，草原是它房。
太阳笑眯眯，云朵轻飘扬，
小马悠悠跑，心儿多欢畅。

河边弯弯饮清水，波光映它靓影长，
花儿点头迎风笑，小马世界美如画框。
朋友小兔跳跳来，小鸟啾啾话家常，
大伙儿一起乐悠悠，草原生活甜如糖。



'小马儿小马儿，清晨蹄声忙，\n踏上绿茵茵，草原是它房。\n太阳笑眯眯，云朵轻飘扬，\n小马悠悠跑，心儿多欢畅。\n\n河边弯弯饮清水，波光映它靓影长，\n花儿点头迎风笑，小马世界美如画框。\n朋友小兔跳跳来，小鸟啾啾话家常，\n大伙儿一起乐悠悠，草原生活甜如糖。'

In [22]:
messages

[{'role': 'system',
  'content': '你是强大的写作助手。\n\n你必须遵循以下约束来完成任务:\n1. 直接输出你的结果，不要评论，不要啰嗦\n2. 使用markdown格式输出\n\n**你的任务是:**\n帮我写一首关于小马的四句儿歌\n'},
 {'role': 'user', 'content': '请你开始'},
 {'role': 'assistant',
  'content': '小马儿小马儿，蹄儿踏踏响，\n草原上奔跑，快乐无边疆。\n尾巴摇摇，鬃毛闪闪亮，\n朋友遍天下，笑声多欢畅。'},
 {'role': 'user', 'content': '改为8句吧'},
 {'role': 'assistant',
  'content': '小马儿小马儿，清晨蹄声忙，\n踏上绿茵茵，草原是它房。\n太阳笑眯眯，云朵轻飘扬，\n小马悠悠跑，心儿多欢畅。\n\n河边弯弯饮清水，波光映它靓影长，\n花儿点头迎风笑，小马世界美如画框。\n朋友小兔跳跳来，小鸟啾啾话家常，\n大伙儿一起乐悠悠，草原生活甜如糖。'}]

In [23]:
chat(qwen, "类似西方的诗歌那样风格", messages, k=3, model="qwen2-1.5b-instruct")

In meadows of gold, where the dawn gently tiptoes,
A little pony trots, its mane in the morning breeze flows.
Hooves create a melody, in rhythm with the gale,
Through the emerald valley, it weaves a carefree tale.

With a tail that sways, in harmony so rare,
Its coat shines like silver, under the sun's tender stare.
The pony explores, with a spirit bold and free,
Friendship's bonds it forges, in this vast sanctuary.

The willows bow, as it grazes by the stream,
Reflecting in waters, a picture like a dream.
Rabbit hops nearby, birds sing in the trees,
Nature's symphony, a choir for the pony's ease.

As twilight descends, stars begin to appear,
The pony rests, under the moon's watchful leer.
In dreams it gallops, through enchanted forests deep,
A western ballad, in the heart of the wild it'll keep.



"In meadows of gold, where the dawn gently tiptoes,\nA little pony trots, its mane in the morning breeze flows.\nHooves create a melody, in rhythm with the gale,\nThrough the emerald valley, it weaves a carefree tale.\n\nWith a tail that sways, in harmony so rare,\nIts coat shines like silver, under the sun's tender stare.\nThe pony explores, with a spirit bold and free,\nFriendship's bonds it forges, in this vast sanctuary.\n\nThe willows bow, as it grazes by the stream,\nReflecting in waters, a picture like a dream.\nRabbit hops nearby, birds sing in the trees,\nNature's symphony, a choir for the pony's ease.\n\nAs twilight descends, stars begin to appear,\nThe pony rests, under the moon's watchful leer.\nIn dreams it gallops, through enchanted forests deep,\nA western ballad, in the heart of the wild it'll keep."

In [24]:
messages

[{'role': 'system',
  'content': '你是强大的写作助手。\n\n你必须遵循以下约束来完成任务:\n1. 直接输出你的结果，不要评论，不要啰嗦\n2. 使用markdown格式输出\n\n**你的任务是:**\n帮我写一首关于小马的四句儿歌\n'},
 {'role': 'user', 'content': '请你开始'},
 {'role': 'assistant',
  'content': '小马儿小马儿，蹄儿踏踏响，\n草原上奔跑，快乐无边疆。\n尾巴摇摇，鬃毛闪闪亮，\n朋友遍天下，笑声多欢畅。'},
 {'role': 'user', 'content': '改为8句吧'},
 {'role': 'assistant',
  'content': '小马儿小马儿，清晨蹄声忙，\n踏上绿茵茵，草原是它房。\n太阳笑眯眯，云朵轻飘扬，\n小马悠悠跑，心儿多欢畅。\n\n河边弯弯饮清水，波光映它靓影长，\n花儿点头迎风笑，小马世界美如画框。\n朋友小兔跳跳来，小鸟啾啾话家常，\n大伙儿一起乐悠悠，草原生活甜如糖。'},
 {'role': 'user', 'content': '类似西方的诗歌那样风格'},
 {'role': 'assistant',
  'content': "In meadows of gold, where the dawn gently tiptoes,\nA little pony trots, its mane in the morning breeze flows.\nHooves create a melody, in rhythm with the gale,\nThrough the emerald valley, it weaves a carefree tale.\n\nWith a tail that sways, in harmony so rare,\nIts coat shines like silver, under the sun's tender stare.\nThe pony explores, with a spirit bold and free,\nFriendship's bonds it forges, in this vast sanctuary.\n\nTh